In [17]:
import pandas as pd
import numpy as np
import sqlite3

In [18]:
def load_day(day):
    header = ['timestamp', 'line_id', 'direction', 'jrny_patt_id', 'time_frame', 'journey_id', 'operator', 
              'congestion', 'lon', 'lat', 'delay', 'block_id', 'vehicle_id', 'stop_id', 'at_stop']
    types = {'timestamp': np.int64,
             'journey_id': np.int32,
             'congestion': np.int8,
             'lon': np.float64,
             'lat': np.float64,
             'delay': np.int8,
             'vehicle_id': np.int32,
             'at_stop': np.int8}
    file_name = 'data/siri.201301{0:02d}.csv'.format(day)
    df = pd.read_csv(file_name, header=None, names=header, dtype=types, parse_dates=['time_frame'], infer_datetime_format=True)
    null_replacements = {'line_id': 0, 'stop_id': 0}
    df = df.fillna(value=null_replacements)
    df['line_id'] = df['line_id'].astype(np.int32)
    df['stop_id'] = df['stop_id'].astype(np.int32)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='us')
    return df


In [19]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    
    Taken from here: https://stackoverflow.com/questions/29545704/fast-haversine-approximation-python-pandas#29546836
    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    #c = 2 * np.arcsin(np.sqrt(a))
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1.0 - a))
    meters = 6372000.0 * c
    return meters

In [20]:
def calculate_durations(data_frame, vehicle_id):
    one_second = np.timedelta64(1000000000, 'ns')
    dv = data_frame[data_frame['vehicle_id']==vehicle_id]
    ts = dv.timestamp.values
    dtd = ts[1:] - ts[:-1]
    dt = np.zeros(len(dtd) + 1)
    dt[1:] = dtd / one_second
    return dt

In [21]:
def calculate_distances(data_frame, vehicle_id):
    dv = data_frame[data_frame['vehicle_id']==vehicle_id]
    lat = dv.lat.values
    lon = dv.lon.values
    dxm = haversine_np(lon[1:], lat[1:], lon[:-1], lat[:-1])
    dx = np.zeros(len(dxm) + 1)
    dx[1:] = dxm
    return dx

In [22]:
def filter_columns(df):
    columns = ['timestamp', 'direction', 'journey_id', 'congestion', 'lon', 'lat', 'delay', 'vehicle_id', 'stop_id', 'at_stop']
    return df[columns]

In [1]:
def create_table(conn):
    c = conn.cursor()
    c.execute('''CREATE TABLE locations (timestamp, direction, journey_id, congestion, lon, lat, vehicle_id, stop_id, at_stop)''')
    conn.commit()

In [25]:
days = None
for d in range(31):
    day = filter_columns(load_day(d+1))
    day['dt'] = 0.0
    day['dx'] = 0.0
    day['speed'] = 0.0    
    if days is None:
        days = day
    else:
        days = days.append(day)
    

In [26]:
days.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44455133 entries, 0 to 603920
Data columns (total 13 columns):
timestamp     datetime64[ns]
direction     int64
journey_id    int32
congestion    int8
lon           float64
lat           float64
delay         int8
vehicle_id    int32
stop_id       int32
at_stop       int8
dt            float64
dx            float64
speed         float64
dtypes: datetime64[ns](1), float64(5), int32(3), int64(1), int8(3)
memory usage: 3.3 GB


In [27]:
vehicles = days['vehicle_id'].unique()

In [28]:
for v in vehicles:
    vehicle_selector = days['vehicle_id']==v
    days.loc[vehicle_selector,'dt'] = calculate_durations(days, v)
    days.loc[vehicle_selector,'dx'] = calculate_distances(days, v)

speed_selector = days['dt'] > 0
days.loc[speed_selector,'speed'] = days[speed_selector].dx / days[speed_selector].dt * 3.6

# Filter invalid points (speeds over 100 km/h)
days = days[days['speed'] < 100.0]


KeyboardInterrupt: 

In [12]:
days.head(10)

,timestamp,direction,journey_id,congestion,lon,lat,delay,vehicle_id,stop_id,at_stop,dt,dx,speed
0,2013-01-02 00:00:01,0,16353,0,-6.323346,53.295563,-74,33491,6132,1,0.0,0.0,0.0
1,2013-01-02 00:00:01,0,15893,0,-6.297000,53.347950,-92,33359,1575,0,0.0,0.0,0.0
2,2013-01-02 00:00:01,0,16049,0,-6.278083,53.416634,-42,38072,6229,0,0.0,0.0,0.0
3,2013-01-02 00:00:01,0,15751,0,-6.278600,53.417133,0,33455,6317,1,0.0,0.0,0.0
4,2013-01-02 00:00:03,0,16312,0,-6.256236,53.350414,-54,38020,621,1,0.0,0.0,0.0
5,2013-01-02 00:00:03,0,13342,0,-6.115118,53.493210,-93,33377,3725,0,0.0,0.0,0.0
6,2013-01-02 00:00:03,0,13094,0,-6.277900,53.416451,0,33160,281,0,0.0,0.0,0.0
7,2013-01-02 00:00:03,0,16074,0,-6.110153,53.202221,-90,33348,4132,1,0.0,0.0,0.0
8,2013-01-02 00:00:03,0,15552,0,-6.254734,53.355618,-19,33119,5052,0,0.0,0.0,0.0
9,2013-01-02 00:00:03,0,15561,0,-6.261019,53.274021,-1,33425,2971,0,0.0,0.0,0.0


In [13]:
days[days['vehicle_id']==33491].head(20)

,timestamp,direction,journey_id,congestion,lon,lat,delay,vehicle_id,stop_id,at_stop,dt,dx,speed
0,2013-01-02 00:00:01,0,16353,0,-6.323346,53.295563,-74,33491,6132,1,0.0,0.000000,0.000000
103,2013-01-02 00:00:17,0,16353,0,-6.323346,53.295563,-14,33491,6132,1,16.0,0.000000,0.000000
136,2013-01-02 00:00:21,0,16353,0,-6.323346,53.295563,-14,33491,6132,1,4.0,0.000000,0.000000
227,2013-01-02 00:00:42,0,16353,0,-6.323346,53.295563,-14,33491,6132,1,21.0,0.000000,0.000000
323,2013-01-02 00:01:02,0,16353,0,-6.323346,53.295563,-14,33491,6132,1,20.0,0.000000,0.000000
430,2013-01-02 00:01:17,0,16353,0,-6.320733,53.296974,-14,33491,4861,0,15.0,234.072988,56.177517
454,2013-01-02 00:01:21,0,16353,0,-6.320733,53.296974,-14,33491,4861,0,4.0,0.000000,0.000000
548,2013-01-02 00:01:41,0,16353,0,-6.320733,53.296974,-14,33491,4861,0,20.0,0.000000,0.000000
638,2013-01-02 00:02:01,0,16353,0,-6.320733,53.296974,-14,33491,4861,0,20.0,0.000000,0.000000
845,2013-01-02 00:02:42,0,16353,0,-6.269917,53.318066,0,33491,4862,1,20.0,321.906731,57.943211


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1667864 entries, 0 to 1687411
Data columns (total 13 columns):
timestamp     1667864 non-null datetime64[ns]
direction     1667864 non-null int64
journey_id    1667864 non-null int32
congestion    1667864 non-null int8
lon           1667864 non-null float64
lat           1667864 non-null float64
delay         1667864 non-null int8
vehicle_id    1667864 non-null int32
stop_id       1667864 non-null int32
at_stop       1667864 non-null int8
dt            1667864 non-null float64
dx            1667864 non-null float64
speed         1667864 non-null float64
dtypes: datetime64[ns](1), float64(5), int32(3), int64(1), int8(3)
memory usage: 125.7 MB


In [15]:
# Number of different vehicles
len(days.groupby('vehicle_id'))

817

In [16]:
# Number of different stops
len(days.groupby('stop_id'))

4713

In [ ]:
days.to_csv("data/201301.csv", index=False)